# Imports and Function Definitions

## Imports

In [ ]:
'''
Train and test various models on three separate QCHAT-10 data sets; perform feature selection analysis
Author: Lydia Sollis
Date: Mar 25 2024
'''

# Install necessary packages not included in the environment
!pip install pyreadstat

# Standard libraries for data manipulation and mathematical operations
import os
import math
import numpy as np 
import pandas as pd

# Data loading and transformation
import pyreadstat
from scipy.io import arff

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing tools from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold, cross_validate, cross_val_predict

# Machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Feature selection tools
from sklearn.feature_selection import RFE, SequentialFeatureSelector

# Metrics for model evaluation
import sklearn.metrics as metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, roc_auc_score, 
    confusion_matrix, classification_report, roc_curve, make_scorer,
    ConfusionMatrixDisplay, balanced_accuracy_score
)

# Configure pandas to raise an exception instead of silently downcasting types
pd.set_option('future.no_silent_downcasting', True) 

# Function to list all files under the input directory in a Kaggle environment
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Functions

In [ ]:
# Custom scorer for specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp)

# Function to ensure columns of different datasets are in the same order prior to train/test
def check_and_reorder_columns(df_source, df_target):
    if (df_source.columns == df_target.columns).all():
        print("Columns are the same.")
        return df_target
    else:
        print("Columns are not the same. Reordering...")
        return df_target[df_source.columns]

# Load / Clean Data

## New Zealand Dataset

In [ ]:
# Load file
path = '/kaggle/input/autism-screening-for-toddlers/Toddler Autism dataset July 2018.csv'
data = pd.read_csv(path)
# data, meta = arff.loadarff(path)

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data)

# If the data contains byte strings, convert them to regular strings
df = df.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# Check the shape of the DataFrame before dropping duplicates
print("Shape of DataFrame before dropping duplicates:", df.shape)

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Check the shape of the DataFrame after dropping duplicates
print("Shape of DataFrame after dropping duplicates:", df.shape)

print(df.head())
print(df.describe())
print(df.info())

# Rename misspelled / unclear columns
# family_pdd = direct family member with pervasive developmental disorder
# Five disorders are identified under the category of pervasive developmental disorders: (1) autistic disorder, (2) Rett's disorder, (3) childhood disintegrative disorder, (4) Asperger's syndrome, and (5) pervasive developmental disorder-not otherwise specified, or PDD-NOS (DSM-IV-TR, 2000).

df = df.rename(columns ={
    'A1' : 'A1_Score',
    'A2' : 'A2_Score',
    'A3' : 'A3_Score',
    'A4' : 'A4_Score',
    'A5' : 'A5_Score',
    'A6' : 'A6_Score',
    'A7' : 'A7_Score',
    'A8' : 'A8_Score',
    'A9' : 'A9_Score',
    'A10' : 'A10_Score',
    'Age_Mons' : 'age_months',
    'Class/ASD Traits ' : 'Class/ASD',
    'Family_mem_with_ASD' : 'family_pdd',
    'Jaundice' : 'jaundice',
    'Ethnicity' : 'ethnicity',
    'Qchat-10-Score' : 'result',
    'Sex' : 'gender',
    'Who completed the test' : 'relation'
})

list(df.columns)

In [ ]:
# print unique values for each column
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

In [ ]:
# For a single column
mean_val = df["age_months"].mean()
std_val = df["age_months"].std()
min_val = df["age_months"].min()
max_val = df["age_months"].max()

print("Mean:", mean_val)
print("Standard Deviation:", std_val)
print("Min:", min_val)
print("Max:", max_val)

In [ ]:
# Replace text objects with boolean values
for column in ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']:  # Add all your column names here
    df[column] = df[column].replace({1: True, 0: False}).infer_objects()

for column in ['jaundice', 'family_pdd']:  # Add all your column names here
    df[column] = df[column].replace({'yes': True, 'no': False}).infer_objects()

df['Class/ASD'] = df['Class/ASD'].replace({'Yes': 1, 'No': 0}).infer_objects()

# Male gender represented as 'True' and female represented as 'False'
df['gender'] = df['gender'].replace({'m': True, 'f': False}).infer_objects()
    
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

print(df.info())

In [ ]:
# Find count of null values
print(df.isnull().sum())

sns.heatmap(df.isnull(), cbar=False)

# Calculate the mean of the 'age_months' column
age_months_mean = df['age_months'].mean()

# Fill missing values in 'age_months' with the calculated mean
df['age_months'] = df['age_months'].fillna(age_months_mean)

In [ ]:
# drop irrelevant columns / columns not in common between all datasets / 
print("Current columns in the DataFrame:", df.columns.tolist())
df_cleaned = df.drop(['Case_No', 'result', 'relation', 'ethnicity', 'jaundice'], axis=1)
df_cleaned.head()

# Check the result
print(df_cleaned.shape)

In [ ]:
import pandas as pd
from scipy import stats

# Example dataframe
# df = pd.DataFrame({
#     "age": [...],
#     "result": [...]
# })

# Split into groups
group0 = df.loc[df["result"] == 0, "age_months"]
group1 = df.loc[df["result"] == 1, "age_months"]

# Independent samples t-test
t_stat, p_val_ttest = stats.ttest_ind(group0, group1, equal_var=False)  # Welch’s t-test
print("T-test results:")
print("t-statistic =", t_stat, "p-value =", p_val_ttest)

# Mann–Whitney U test (non-parametric)
u_stat, p_val_mwu = stats.mannwhitneyu(group0, group1, alternative="two-sided")
print("\nMann-Whitney U test results:")
print("U-statistic =", u_stat, "p-value =", p_val_mwu)

In [ ]:
# Iterate over each column in the DataFrame
for column in df_cleaned.columns:
    # Check data type of the column
    if df_cleaned[column].dtype == 'object':  # Categorical data
        # Generate a bar plot for categorical data
        df_cleaned[column].value_counts().plot(kind='bar', title=f'Distribution of {column}')
        plt.xticks(rotation=45)  # Adjust rotation if necessary
    elif df_cleaned[column].dtype == 'bool' or (df_cleaned[column].dtype == 'int64' and df_cleaned[column].nunique() == 2):
        # Handle binary data specially
        df_cleaned[column].astype(int).plot(kind='hist', bins=[-0.5, 0.5, 1.5], title=f'Distribution of {column}', rwidth=0.8)
        plt.xticks([0, 1])  # Set x-ticks specifically for binary data
    elif df_cleaned[column].dtype in ['float64', 'int64']:
        # Handle float and integer data types for non-binary data
        df_cleaned[column].plot(kind='hist', bins=10, title=f'Distribution of {column}', rwidth=0.8)
    else:
        print(f"Skipping {column} as it is not suitable for histogram plotting.")
    
    plt.show()  # Ensure the plot is shown after each column

In [ ]:
class_asd_counts = df_cleaned['Class/ASD'].value_counts()
print(class_asd_counts)

for column in df_cleaned.columns:
    print(f"{column}: {df_cleaned[column].unique()}")

print(df_cleaned.info())

df_cleaned_nz = df_cleaned

## Polish Dataset

In [ ]:
# load polish dataset
df, meta = pyreadstat.read_sav('/kaggle/input/qchat-mendeley-polish-toddlers/QCHAT_dataset2 mendeley.sav')

# Check the shape of the DataFrame before dropping duplicates
print("Shape of DataFrame before dropping duplicates:", df.shape)

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Check the shape of the DataFrame after dropping duplicates
print("Shape of DataFrame after dropping duplicates:", df.shape)

print(df.head())
print(df.describe())
print(df.info())

df = df.rename(columns ={
    'qchat1recode' : 'A1_Score',
    'qchat2recode' : 'A2_Score',
    'qchat5recode' : 'A3_Score',
    'qchat6recode' : 'A4_Score',
    'qchat9recode' : 'A5_Score',
    'qchat10recode' : 'A6_Score',
    'qchat15recode' : 'A7_Score',
    'qchat17recode' : 'A8_Score',
    'qchat19recode' : 'A9_Score',
    'qchat25recode' : 'A10_Score',
    'age' : 'age_months',
    'group' : 'Class/ASD',
    'sibling_withASD' : 'family_pdd',
    'Ethnicity' : 'ethnicity',
    'Sum_QCHAT' : 'result',
    'sex' : 'gender'
})

list(df.columns)

In [ ]:
# print unique values for each column
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

In [ ]:
# For a single column
mean_val = df["age_months"].mean()
std_val = df["age_months"].std()
min_val = df["age_months"].min()
max_val = df["age_months"].max()

print("Mean:", mean_val)
print("Standard Deviation:", std_val)
print("Min:", min_val)
print("Max:", max_val)

In [ ]:
# Replace text objects with boolean values
#for column in ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score']:  # Add all your column names here
#    df[column] = df[column].replace({4: False, 3: False, 2:True, 1: True, 0:True}).infer_objects()
# Update A10 scoring to reflect the New Zealand dataset
for column in ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score']:  # Add all your column names here
    df[column] = df[column].replace({4: True, 3: True, 2:True, 1: False, 0:False}).infer_objects()
df['A10_Score'] = df['A10_Score'].replace({4: True, 3: True, 2:True, 1: False, 0:False}).infer_objects()

# for column in ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score']:  # Add all your column names here
#     df[column] = df[column].replace({4: False, 3: False, 2:True, 1: True, 0:True}).infer_objects()

for column in ['family_pdd']:  # Add all your column names here
    df[column] = df[column].replace({1: True, 0: False, math.nan:False}).infer_objects()

df['Class/ASD'] = df['Class/ASD'].replace({1: 1, 7: 0}).infer_objects()

# Male gender represented as 'True' and female represented as 'False'
df['gender'] = df['gender'].replace({1: True, 2: False}).infer_objects()
    
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

print(df.info())

In [ ]:
# Find count of null values
print(df.isnull().sum())

sns.heatmap(df.isnull(), cbar=False)

In [ ]:
# drop irrelevant columns and columns not shared with the other dataset
print("Current columns in the DataFrame:", df.columns.tolist())
df_cleaned = df.drop(['child_id', 'preterm', 'birthweight', 'result', 'siblings_yesno', 'siblings_number', 'mothers_education', 'qchat13recode', 'qchat21recode', 'qchat23recode', 'qchat3recode', 'qchat7recode', 'qchat11recode', 'qchat4recode', 'qchat8recode', 'qchat12recode', 'qchat14recode', 'qchat18recode', 'qchat16recode', 'qchat20recode', 'qchat22recode', 'qchat24recode'], axis=1)
df_cleaned.head()

# Check the result
print(df_cleaned.shape)
print("Current columns in the DataFrame:", df_cleaned.columns.tolist())

In [ ]:
# Iterate over each column in the DataFrame
for column in df_cleaned.columns:
    # Check data type of the column
    if df_cleaned[column].dtype == 'object':  # Categorical data
        # Generate a bar plot for categorical data
        df_cleaned[column].value_counts().plot(kind='bar', title=f'Distribution of {column}')
        plt.xticks(rotation=45)  # Adjust rotation if necessary
    elif df_cleaned[column].dtype == 'bool' or (df_cleaned[column].dtype == 'int64' and df_cleaned[column].nunique() == 2):
        # Handle binary data specially
        df_cleaned[column].astype(int).plot(kind='hist', bins=[-0.5, 0.5, 1.5], title=f'Distribution of {column}', rwidth=0.8)
        plt.xticks([0, 1])  # Set x-ticks specifically for binary data
    elif df_cleaned[column].dtype in ['float64', 'int64']:
        # Handle float and integer data types for non-binary data
        df_cleaned[column].plot(kind='hist', bins=10, title=f'Distribution of {column}', rwidth=0.8)
    else:
        print(f"Skipping {column} as it is not suitable for histogram plotting.")
    
    plt.show()  # Ensure the plot is shown after each column

In [ ]:
# drop irrelevant columns and columns not shared with the other dataset
print("Current columns in the DataFrame:", df.columns.tolist())
df_cleaned = df.drop(['child_id', 'preterm', 'birthweight', 'result', 'siblings_yesno', 'siblings_number', 'mothers_education', 'qchat13recode', 'qchat21recode', 'qchat23recode', 'qchat3recode', 'qchat7recode', 'qchat11recode', 'qchat4recode', 'qchat8recode', 'qchat12recode', 'qchat14recode', 'qchat18recode', 'qchat16recode', 'qchat20recode', 'qchat22recode', 'qchat24recode'], axis=1)
df_cleaned.head()

# Check the result
print(df_cleaned.shape)
print("Current columns in the DataFrame:", df_cleaned.columns.tolist())

In [ ]:
class_asd_counts = df_cleaned['Class/ASD'].value_counts()
print(class_asd_counts)

for column in df_cleaned.columns:
    print(f"{column}: {df_cleaned[column].unique()}")

print(df_cleaned.info())

df_cleaned_polish = df_cleaned

## Saudi Dataset

In [ ]:
# Load file
path = '/kaggle/input/asd-screening-data-for-toddlers-in-saudi-arabia/Autism Spectrum Disorder Screening Data for Toddlers in Saudi Arabia Data Set.csv'
data = pd.read_csv(path)
# data, meta = arff.loadarff(path)

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data)

# If the data contains byte strings, convert them to regular strings
df = df.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# Check the shape of the DataFrame before dropping duplicates
print("Shape of DataFrame before dropping duplicates:", df.shape)

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Check the shape of the DataFrame after dropping duplicates
print("Shape of DataFrame after dropping duplicates:", df.shape)

print(df.head())
print(df.describe())
print(df.info())

# Rename misspelled / unclear columns
# family_pdd = direct family member with pervasive developmental disorder
# Five disorders are identified under the category of pervasive developmental disorders: (1) autistic disorder, (2) Rett's disorder, (3) childhood disintegrative disorder, (4) Asperger's syndrome, and (5) pervasive developmental disorder-not otherwise specified, or PDD-NOS (DSM-IV-TR, 2000).

df = df.rename(columns ={
    'A1' : 'A1_Score',
    'A2' : 'A2_Score',
    'A3' : 'A3_Score',
    'A4' : 'A4_Score',
    'A5' : 'A5_Score',
    'A6' : 'A6_Score',
    'A7' : 'A7_Score',
    'A8' : 'A8_Score',
    'A9' : 'A9_Score',
    'A10' : 'A10_Score',
    'Age' : 'age_months',
    'Class' : 'Class/ASD',
    'Family member with ASD history' : 'family_pdd',
    'Jaundice' : 'jaundice',
    'Ethnicity' : 'ethnicity',
    'Screening Score' : 'result',
    'Gender' : 'gender',
    'Who is completing the test' : 'relation'
})

list(df.columns)

In [ ]:
# print unique values for each column
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

In [ ]:
# For a single column
mean_val = df["age_months"].mean()
std_val = df["age_months"].std()
min_val = df["age_months"].min()
max_val = df["age_months"].max()

print("Mean:", mean_val)
print("Standard Deviation:", std_val)
print("Min:", min_val)
print("Max:", max_val)

In [ ]:
import pandas as pd
from scipy import stats

# Example dataframe
# df = pd.DataFrame({
#     "age": [...],
#     "result": [...]
# })

# Split into groups
group0 = df.loc[df["result"] == 0, "age_months"]
group1 = df.loc[df["result"] == 1, "age_months"]

# Independent samples t-test
t_stat, p_val_ttest = stats.ttest_ind(group0, group1, equal_var=False)  # Welch’s t-test
print("T-test results:")
print("t-statistic =", t_stat, "p-value =", p_val_ttest)

# Mann–Whitney U test (non-parametric)
u_stat, p_val_mwu = stats.mannwhitneyu(group0, group1, alternative="two-sided")
print("\nMann-Whitney U test results:")
print("U-statistic =", u_stat, "p-value =", p_val_mwu)


In [ ]:
# Replace text objects with boolean values
for column in ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']:  # Add all your column names here
    df[column] = df[column].replace({1: True, 0: False}).infer_objects()

for column in ['family_pdd']:  # Add all your column names here
    df[column] = df[column].replace({'Yes': True, 'No': False}).infer_objects()

df['Class/ASD'] = df['Class/ASD'].replace({1: 1, 0: 0}).infer_objects()

# Male gender represented as 'True' and female represented as 'False'
df['gender'] = df['gender'].replace({'Male': True, 'Female': False}).infer_objects()
    
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

print(df.info())

In [ ]:
# Find count of null values
print(df.isnull().sum())

sns.heatmap(df.isnull(), cbar=False)

In [ ]:
# drop irrelevant columns and columns not shared with the other dataset
print("Current columns in the DataFrame:", df.columns.tolist())
df_cleaned = df.drop(['result', 'Region', 'relation'], axis=1)
df_cleaned.head()

# Check the result
print(df_cleaned.shape)
print("Current columns in the DataFrame:", df_cleaned.columns.tolist())

In [ ]:
# Iterate over each column in the DataFrame
for column in df_cleaned.columns:
    # Check data type of the column
    if df_cleaned[column].dtype == 'object':  # Categorical data
        # Generate a bar plot for categorical data
        df_cleaned[column].value_counts().plot(kind='bar', title=f'Distribution of {column}')
        plt.xticks(rotation=45)  # Adjust rotation if necessary
    elif df_cleaned[column].dtype == 'bool' or (df_cleaned[column].dtype == 'int64' and df_cleaned[column].nunique() == 2):
        # Handle binary data specially
        df_cleaned[column].astype(int).plot(kind='hist', bins=[-0.5, 0.5, 1.5], title=f'Distribution of {column}', rwidth=0.8)
        plt.xticks([0, 1])  # Set x-ticks specifically for binary data
    elif df_cleaned[column].dtype in ['float64', 'int64']:
        # Handle float and integer data types for non-binary data
        df_cleaned[column].plot(kind='hist', bins=10, title=f'Distribution of {column}', rwidth=0.8)
    else:
        print(f"Skipping {column} as it is not suitable for histogram plotting.")
    
    plt.show()  # Ensure the plot is shown after each column

In [ ]:
class_asd_counts = df_cleaned['Class/ASD'].value_counts()
print(class_asd_counts)

for column in df_cleaned.columns:
    print(f"{column}: {df_cleaned[column].unique()}")

print(df_cleaned.info())

df_cleaned_saudi = df_cleaned

# Preprocessing

In [ ]:
# Separating features and labels
# nz dataset
X_nz = df_cleaned_nz.drop('Class/ASD', axis=1)  # Features
y_nz = df_cleaned_nz['Class/ASD']  # Labels

# Polish dataset
X_polish = df_cleaned_polish.drop('Class/ASD', axis=1)  # Features
y_polish = df_cleaned_polish['Class/ASD']  # Labels

# Saudi dataset
X_saudi = df_cleaned_saudi.drop('Class/ASD', axis=1)  # Features
y_saudi = df_cleaned_saudi['Class/ASD']  # Labels

# Using only decision tree-based models, so scaling is unnecessary

# Baseline Model Testing / Feature Importance Exploration

## Polish Dataset

### XGBoost

In [ ]:
# Initialize the XGBoost model
xgb_model_polish = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the number of folds for cross-validation
k = 5  # Number of folds. k=5 or k=10 is often recommended
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Specify the parameter grid to search
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [1, 3, 5, 7, 9, 10],  # Typical values for depth in XGBoost
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Learning rate or eta
    'subsample': [0.5, 0.75, 1],  # Subsample ratio of the training instances
    'colsample_bytree': [0.5, 0.7, 1],  # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],  # Minimum loss reduction required to make a further partition
    'reg_lambda': [1, 2, 3, 4, 5],  # L2 regularization term on weights
    'reg_alpha': [0, 0.1, 0.2]  # L1 regularization term on weights
}

# Define different scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_model_polish,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_polish, y_polish)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

In [ ]:
# Initialize the XGBoost model with relevant hyperparameters
xgb_model_polish = XGBClassifier(
    n_estimators=400,
    max_depth=1,
    learning_rate=0.3,
    subsample=1,
    colsample_bytree=1,
    gamma=0.5,
    reg_lambda=2,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Compute the correlation matrix
correlation_matrix = X_polish.corr()

# Plot the correlation matrix heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix Heatmap Polish')
plt.show()

# Perform cross-validation and return estimators
cv_results = cross_validate(xgb_model_polish, X_polish, y_polish, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix
y_pred = cross_val_predict(xgb_model_polish, X_polish, y_polish, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_polish, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['test_balanced_accuracy', 'test_sensitivity', 'test_specificity', 'test_roc_auc']:
    print(f"{metric.replace('test_', '').replace('_', ' ').capitalize()} across {k} folds: {np.mean(cv_results[metric]):.2f} ± {np.std(cv_results[metric]):.2f}")

# Collect all feature importances
feature_importances = np.zeros(X_polish.shape[1])
for estimator in cv_results['estimator']:
    feature_importances += estimator.feature_importances_

# Average feature importances
feature_importances /= k

features = X_polish.columns

# Print feature importances in order
print("Feature Importances:")
sorted_indices = np.argsort(feature_importances)[::-1]  # Sort indices in descending order of importance
for i in sorted_indices:
    print(f"{features[i]}: {feature_importances[i]}")

# Plot feature importances
indices = np.argsort(feature_importances)
plt.figure(figsize=(10, 8))
plt.title('Feature Importances')
plt.barh(range(len(indices)), feature_importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# Plot the confusion matrix
plt.figure()
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Cutoff for positive prediction

In [ ]:
# Custom scorer for specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp)

# Assuming X_polish and y_polish are your feature matrix and target vector

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_polish = XGBClassifier(
    n_estimators=400,
    max_depth=1,
    learning_rate=0.3,
    subsample=1,
    colsample_bytree=1,
    gamma=0.5,
    reg_lambda=2,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Perform cross-validation and return estimators
cv_results = cross_validate(xgb_model_polish, X_polish, y_polish, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix with custom threshold
y_pred_proba = cross_val_predict(xgb_model_polish, X_polish, y_polish, cv=cv, method='predict_proba')[:, 1]
y_pred = (y_pred_proba >= 0.3).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(y_polish, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['test_balanced_accuracy', 'test_sensitivity', 'test_specificity', 'test_roc_auc']:
    print(f"{metric.replace('test_', '').replace('_', ' ').capitalize()} across {k} folds: {np.mean(cv_results[metric]):.2f} ± {np.std(cv_results[metric]):.2f}")

# Plot the confusion matrix
plt.figure()
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### Recursive Feature Elimination

In [ ]:
# Initialize the XGBoost model with relevant hyperparameters
xgb_model_polish = XGBClassifier(
    n_estimators=400,
    max_depth=1,
    learning_rate=0.3,
    subsample=1,
    colsample_bytree=1,
    gamma=0.5,
    reg_lambda=2,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define different scoring metrics
scoring = {
    'sensitivity': 'recall',
    'specificity': make_scorer(specificity_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Initialize lists to store metrics for each number of features
metrics_history = {
    'sensitivity': [],
    'specificity': [],
    'balanced_accuracy': []
}

num_features = X_polish.shape[1]
features_list = X_polish.columns.tolist()  # Store initial full feature list

# Perform RFE with cross-validation and track metrics
for i in range(num_features, 0, -1):
    rfe = RFE(estimator=xgb_model_polish, n_features_to_select=i, step=1)
    rfe.fit(X_polish, y_polish)
    
    # Current set of selected features
    current_features = list(X_polish.columns[rfe.support_])
    print(f"Features retained ({i}): {current_features}")
    
    # Evaluate metrics for the selected features
    cv_results = cross_validate(rfe.estimator_, rfe.transform(X_polish), y_polish, cv=cv, scoring=scoring)
    
    # Store results for plotting
    for metric in scoring:
        mean_score = cv_results[f'test_{metric}'].mean()
        std_score = cv_results[f'test_{metric}'].std()
        metrics_history[metric].append(mean_score)
        print(f"{metric.capitalize()} with {i} features: {mean_score:.2f} ± {std_score:.2f}")

# Plotting the metrics over the number of features
plt.figure(figsize=(12, 8))
feature_counts = list(range(num_features, 0, -1))
for metric, values in metrics_history.items():
    plt.plot(feature_counts, values, marker='o', linestyle='-', label=metric.capitalize())

plt.title('Polish-Trained Model Performance Across Feature Counts')
plt.xlabel('Number of Features')
plt.ylabel('Model Performance')
plt.legend()
# plt.gca().invert_xaxis()  # Optional: Invert x-axis to show decreasing number of features
plt.grid(True)
# Save the plot to /kaggle/working directory
plt.savefig('/kaggle/working/model_performance_across_feature_counts_polish.png')
plt.show()

#### 4 Features Only

In [ ]:
# Select only the 4 retained features
retained_features = ['A2_Score', 'A3_Score', 'A4_Score', 'A5_Score']
X_polish_reduced = X_polish[retained_features]

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_polish = XGBClassifier(
    n_estimators=400,
    max_depth=1,
    learning_rate=0.3,
    subsample=1,
    colsample_bytree=1,
    gamma=0.5,
    reg_lambda=2,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Perform cross-validation and return estimators
cv_results = cross_validate(xgb_model_polish, X_polish_reduced, y_polish, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix
y_pred = cross_val_predict(xgb_model_polish, X_polish_reduced, y_polish, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_polish, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['test_balanced_accuracy', 'test_sensitivity', 'test_specificity', 'test_roc_auc']:
    print(f"{metric.replace('test_', '').replace('_', ' ').capitalize()} across {k} folds: {np.mean(cv_results[metric]):.2f} ± {np.std(cv_results[metric]):.2f}")

# Plot the confusion matrix
plt.figure()
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Select only the 4 retained features
retained_features = ['A2_Score', 'A3_Score', 'A4_Score', 'A5_Score']
X_polish_reduced = X_polish[retained_features]

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_polish = XGBClassifier(
    n_estimators=400,
    max_depth=1,
    learning_rate=0.3,
    subsample=1,
    colsample_bytree=1,
    gamma=0.5,
    reg_lambda=2,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Perform cross-validation and return estimators
cv_results = cross_validate(xgb_model_polish, X_polish_reduced, y_polish, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix with custom threshold
y_pred_proba = cross_val_predict(xgb_model_polish, X_polish_reduced, y_polish, cv=cv, method='predict_proba')[:, 1]
y_pred = (y_pred_proba >= 0.3).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(y_polish, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['test_balanced_accuracy', 'test_sensitivity', 'test_specificity', 'test_roc_auc']:
    print(f"{metric.replace('test_', '').replace('_', ' ').capitalize()} across {k} folds: {np.mean(cv_results[metric]):.2f} ± {np.std(cv_results[metric]):.2f}")

# Plot the confusion matrix
plt.figure()
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

## New Zealand Dataset

### Decision Tree

In [ ]:
# Define the Decision Tree model
tree_model_nz_paramSearch = DecisionTreeClassifier()

# Specify the parameter grid to search
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [15, 20, 25], # explored range 10 to 50
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

# Define the number of folds for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=tree_model_nz_paramSearch,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_nz, y_nz)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

# Optional: Extract the best model and use it for further analysis or validation
best_tree_model = random_search.best_estimator_

In [ ]:
# Initialize the Decision Tree model
tree_model_nz = DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=1, max_features='sqrt', max_depth=25, criterion='entropy')

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define different scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Perform cross-validation
cv_results = cross_validate(tree_model_nz, X_nz, y_nz, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for confusion matrix
y_pred = cross_val_predict(tree_model_nz, X_nz, y_nz, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_nz, y_pred)
cm_display = ConfusionMatrixDisplay(cm)

# Printing metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['test_accuracy', 'test_precision', 'test_recall', 'test_roc_auc', 'test_specificity']:
    print(f"{metric.capitalize()} across {k} folds: {np.mean(cv_results[metric]):.2f} ± {np.std(cv_results[metric]):.2f}")

### Random Forest

In [ ]:
# Initialize the Random Forest model
forest_model_nz = RandomForestClassifier()

# Define the number of folds for cross-validation
k = 5  # Number of folds. k=5 or k=10 is often recommended
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Specify the parameter grid to search
param_dist = {
    'n_estimators': [100, 200, 300, 400],  # Number of trees in the forest
    'criterion': ['gini', 'entropy'],
    'max_depth': [40, 50, 60, 70], # explored none to 10 to ??
    'min_samples_split': [2, 3, 4], # explored 2 to 10
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],  # Options for the number of features to consider when looking for the best split
    'bootstrap': [True, False]  # Method for sampling data points (with or without replacement)
}

# Define different scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=forest_model_nz,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_nz, y_nz)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

In [ ]:
# Initialize the Random Forest model with specified hyperparameters
forest_model_nz = RandomForestClassifier(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    max_depth=50,
    criterion='gini',
    random_state=42,
    bootstrap=False
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'specificity': make_scorer(specificity_score),  # Include specificity in the scoring dictionary
    'roc_auc': 'roc_auc'
}

# Perform cross-validation and retain the models for feature importance
cv_results = cross_validate(forest_model_nz, X_nz, y_nz, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for confusion matrix
y_pred = cross_val_predict(forest_model_nz, X_nz, y_nz, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_nz, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Printing metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in scoring.keys():
    print(f"{metric.capitalize()} across {k} folds: {np.mean(cv_results['test_' + metric]):.2f} ± {np.std(cv_results['test_' + metric]):.2f}")

### XGBoost

In [ ]:
# Initialize the XGBoost model
xgb_model_nz = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the number of folds for cross-validation
k = 5  # Number of folds. k=5 or k=10 is often recommended
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Specify the parameter grid to search
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 10],  # Typical values for depth in XGBoost
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Learning rate or eta
    'subsample': [0.5, 0.75, 1],  # Subsample ratio of the training instances
    'colsample_bytree': [0.5, 0.7, 1],  # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2, 0.3],  # Minimum loss reduction required to make a further partition
    'reg_lambda': [1, 2, 3],  # L2 regularization term on weights
    'reg_alpha': [0, 0.1, 0.2]  # L1 regularization term on weights
}

# Define different scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_model_nz,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_nz, y_nz)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

In [ ]:
# Assuming X_nz and y_nz are your feature matrix and target vector

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Compute the correlation matrix
correlation_matrix = X_nz.corr()

# Plot the correlation matrix heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix Heatmap New Zealand')
plt.show()

# Perform cross-validation and return estimators
cv_results = cross_validate(xgb_model_nz, X_nz, y_nz, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix
y_pred = cross_val_predict(xgb_model_nz, X_nz, y_nz, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_nz, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['test_accuracy', 'test_precision', 'test_recall', 'test_specificity', 'test_roc_auc']:
    print(f"{metric.capitalize()} across {k} folds: {np.mean(cv_results[metric]):.2f} ± {np.std(cv_results[metric]):.2f}")

# Collect all feature importances
feature_importances = np.zeros(X_nz.shape[1])
for estimator in cv_results['estimator']:
    feature_importances += estimator.feature_importances_

# Average feature importances
feature_importances /= k

features = X_nz.columns

# Print feature importances in order
print("Feature Importances:")
sorted_indices = np.argsort(feature_importances)[::-1]  # Sort indices in descending order of importance
for i in sorted_indices:
    print(f"{features[i]}: {feature_importances[i]}")

# Plot feature importances
indices = np.argsort(feature_importances)
plt.figure(figsize=(10, 8))
plt.title('Feature Importances')
plt.barh(range(len(indices)), feature_importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# Plot the confusion matrix
plt.figure()
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### Recursive Feature Elimination

In [ ]:
# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define different scoring metrics
scoring = {
    'sensitivity': 'recall',
    'specificity': make_scorer(specificity_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'roc_auc': 'roc_auc'  # Adding AUROC
}

# Initialize lists to store metrics for each number of features
metrics_history = {
    'sensitivity': [],
    'specificity': [],
    'balanced_accuracy': [],
    'roc_auc': []  # Adding AUROC to history
}

num_features = X_nz.shape[1]
features_list = X_nz.columns.tolist()  # Store initial full feature list

# Perform RFE with cross-validation and track metrics
for i in range(num_features, 0, -1):
    rfe = RFE(estimator=xgb_model_nz, n_features_to_select=i, step=1)
    rfe.fit(X_nz, y_nz)
    
    # Current set of selected features
    current_features = list(X_nz.columns[rfe.support_])
    print(f"Features retained ({i}): {current_features}")
    
    # Evaluate metrics for the selected features
    cv_results = cross_validate(rfe.estimator_, rfe.transform(X_nz), y_nz, cv=cv, scoring=scoring)
    
    # Store results for plotting
    for metric in ['sensitivity', 'specificity', 'balanced_accuracy']:  # Excluding AUROC from the plot
        mean_score = cv_results[f'test_{metric}'].mean()
        std_score = cv_results[f'test_{metric}'].std()
        metrics_history[metric].append(mean_score)
        print(f"{metric.capitalize()} with {i} features: {mean_score:.2f} ± {std_score:.2f}")
    
    # Also print AUROC separately
    mean_score = cv_results['test_roc_auc'].mean()
    std_score = cv_results['test_roc_auc'].std()
    metrics_history['roc_auc'].append(mean_score)
    print(f"ROC-AUC with {i} features: {mean_score:.2f} ± {std_score:.2f}")

# Plotting the metrics over the number of features (excluding AUROC)
plt.figure(figsize=(12, 8))
feature_counts = list(range(num_features, 0, -1))
for metric, values in metrics_history.items():
    if metric != 'roc_auc':  # Exclude AUROC from the plot
        plt.plot(feature_counts, values, marker='o', linestyle='-', label=metric.capitalize())

plt.title('New Zealand-Trained Model Performance Across Feature Counts')
plt.xlabel('Number of Features')
plt.ylabel('Model Performance')
plt.legend()
# plt.gca().invert_xaxis()  # Optional: Invert x-axis to show decreasing number of features
plt.grid(True)
# Save the plot to /kaggle/working directory
plt.savefig('/kaggle/working/model_performance_across_feature_counts_nz.png')
plt.show()

## Saudi Dataset

### Decision Tree

In [ ]:
# Define the Decision Tree model
tree_model_saudi_paramSearch = DecisionTreeClassifier()

# Specify the parameter grid to search
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, 30, 40, 50], # explored range 10 to 50
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

# Define the number of folds for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=tree_model_saudi_paramSearch,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_saudi, y_saudi)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

# Optional: Extract the best model and use it for further analysis or validation
best_tree_model = random_search.best_estimator_

In [ ]:
# Initialize the Decision Tree model
tree_model_saudi = DecisionTreeClassifier(min_samples_split=5, min_samples_leaf=1, max_features=None, max_depth=50, criterion='entropy')

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define different scoring metrics, including specificity
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Perform cross-validation
cv_results = cross_validate(tree_model_saudi, X_saudi, y_saudi, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix
y_pred = cross_val_predict(tree_model_saudi, X_saudi, y_saudi, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_saudi, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in ['accuracy', 'precision', 'recall', 'specificity', 'roc_auc']:  # Remove the redundant 'test_' from the list items
    mean_value = np.mean(cv_results[f'test_{metric}'])
    std_value = np.std(cv_results[f'test_{metric}'])
    print(f"{metric.capitalize()} across {k} folds: {mean_value:.2f} ± {std_value:.2f}")

### Random Forest

In [ ]:
# Initialize the Random Forest model
forest_model_saudi = RandomForestClassifier()

# Define the number of folds for cross-validation
k = 5  # Number of folds. k=5 or k=10 is often recommended
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Specify the parameter grid to search
param_dist = {
    'n_estimators': [50, 75, 100, 125, 150, 175],  # Tested 50 to 400
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15], # Tested none to 5 to 50
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3], # Tested 1 to 4
    'max_features': ['sqrt', 'log2'],  # Options for the number of features to consider when looking for the best split
    'bootstrap': [True, False]  # Method for sampling data points (with or without replacement)
}

# Define different scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'roc_auc': 'roc_auc'
}

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=forest_model_saudi,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_saudi, y_saudi)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

In [ ]:
# Initialize the Random Forest model with specified hyperparameters
forest_model_saudi = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='log2',
    max_depth=10,
    criterion='entropy',
    random_state=42,
    bootstrap=False
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics, including specificity
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Perform cross-validation and retain the models for feature importance
cv_results = cross_validate(forest_model_saudi, X_saudi, y_saudi, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for confusion matrix
y_pred = cross_val_predict(forest_model_saudi, X_saudi, y_saudi, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_saudi, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in scoring.keys():
    if metric != 'roc_auc':  # roc_auc is already formatted in the previous print statement
        mean_score = np.mean(cv_results[f'test_{metric}'])
        std_score = np.std(cv_results[f'test_{metric}'])
        print(f"{metric.capitalize()} across {k} folds: {mean_score:.2f} ± {std_score:.2f}")

### XGBoost

In [ ]:
# Initialize the XGBoost model
xgb_model_saudi = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the number of folds for cross-validation
k = 5  # Number of folds. k=5 or k=10 is often recommended
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Specify the parameter grid to search
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 10],  # Typical values for depth in XGBoost
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Learning rate or eta
    'subsample': [0.5, 0.75, 1],  # Subsample ratio of the training instances
    'colsample_bytree': [0.5, 0.7, 1],  # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2, 0.3],  # Minimum loss reduction required to make a further partition
    'reg_lambda': [1, 2, 3],  # L2 regularization term on weights
    'reg_alpha': [0, 0.1, 0.2]  # L1 regularization term on weights
}

# Define different scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'roc_auc': 'roc_auc'
}

# Set up the randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_model_saudi,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Can be changed according to what metric you care about
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_saudi, y_saudi)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

In [ ]:
# Assuming X_saudi and y_saudi are your feature matrix and target vector

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score),
    'roc_auc': 'roc_auc'
}

# Compute the correlation matrix
correlation_matrix = X_saudi.corr()

# Plot the correlation matrix heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix Heatmap Saudi')
plt.show()

# Perform cross-validation and return estimators
cv_results = cross_validate(xgb_model_saudi, X_saudi, y_saudi, cv=cv, scoring=scoring, return_estimator=True)

# Collect predictions for the confusion matrix
y_pred = cross_val_predict(xgb_model_saudi, X_saudi, y_saudi, cv=cv)

# Compute the confusion matrix
cm = confusion_matrix(y_saudi, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Print metrics
print(f"ROC-AUC across {k} folds: {np.mean(cv_results['test_roc_auc']):.2f} ± {np.std(cv_results['test_roc_auc']):.2f}")
for metric in scoring.keys():
    if metric != 'roc_auc':  # 'roc_auc' does not require 'test_' prefix in print
        print(f"{metric.capitalize()} across {k} folds: {np.mean(cv_results[f'test_{metric}']):.2f} ± {np.std(cv_results[f'test_{metric}']):.2f}")

# Collect all feature importances
feature_importances = np.zeros(X_saudi.shape[1])
for estimator in cv_results['estimator']:
    feature_importances += estimator.feature_importances_

# Average feature importances
feature_importances /= k

# Print feature importances in order
print("Feature Importances:")
sorted_indices = np.argsort(feature_importances)[::-1]  # Sort indices in descending order of importance
for i in sorted_indices:
    print(f"{features[i]}: {feature_importances[i]}")

# Plot feature importances
features = X_saudi.columns
indices = np.argsort(feature_importances)
plt.figure(figsize=(10, 8))
plt.title('Feature Importances')
plt.barh(range(len(indices)), feature_importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# Plot the confusion matrix
plt.figure()
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

### Recursive Feature Elimination

In [ ]:
# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the number of folds for cross-validation
k = 5  # Number of folds
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define different scoring metrics
scoring = {
    'sensitivity': 'recall',
    'specificity': make_scorer(specificity_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'roc_auc': 'roc_auc'  # Adding AUROC
}

# Initialize lists to store metrics for each number of features
metrics_history = {
    'sensitivity': [],
    'specificity': [],
    'balanced_accuracy': [],
    'roc_auc': []  # Adding AUROC to history
}

num_features = X_saudi.shape[1]
features_list = X_saudi.columns.tolist()  # Store initial full feature list

# Perform RFE with cross-validation and track metrics
for i in range(num_features, 0, -1):
    rfe = RFE(estimator=xgb_model_saudi, n_features_to_select=i, step=1)
    rfe.fit(X_saudi, y_saudi)
    
    # Current set of selected features
    current_features = list(X_saudi.columns[rfe.support_])
    print(f"Features retained ({i}): {current_features}")
    
    # Evaluate metrics for the selected features
    cv_results = cross_validate(rfe.estimator_, rfe.transform(X_saudi), y_saudi, cv=cv, scoring=scoring)
    
    # Store results for plotting
    for metric in ['sensitivity', 'specificity', 'balanced_accuracy']:  # Excluding AUROC from the plot
        mean_score = cv_results[f'test_{metric}'].mean()
        std_score = cv_results[f'test_{metric}'].std()
        metrics_history[metric].append(mean_score)
        print(f"{metric.capitalize()} with {i} features: {mean_score:.2f} ± {std_score:.2f}")
    
    # Also print AUROC separately
    mean_score = cv_results['test_roc_auc'].mean()
    std_score = cv_results['test_roc_auc'].std()
    metrics_history['roc_auc'].append(mean_score)
    print(f"ROC-AUC with {i} features: {mean_score:.2f} ± {std_score:.2f}")

# Plotting the metrics over the number of features (excluding AUROC)
plt.figure(figsize=(12, 8))
feature_counts = list(range(num_features, 0, -1))
for metric, values in metrics_history.items():
    if metric != 'roc_auc':  # Exclude AUROC from the plot
        plt.plot(feature_counts, values, marker='o', linestyle='-', label=metric.capitalize())

plt.title('Saudi-Trained Model Performance Across Feature Counts')
plt.xlabel('Number of Features')
plt.ylabel('Model Performance')
plt.legend()
# plt.gca().invert_xaxis()  # Optional: Invert x-axis to show decreasing number of features
plt.grid(True)
# Save the plot to /kaggle/working directory
plt.savefig('/kaggle/working/model_performance_across_feature_counts_saudi.png')
plt.show()

# Maximize Recall Full Featured and Best-Performing Model

## New Zealand Dataset - XGB Model

### Full Feature Model - train on New Zealand and validate on Saudi

#### 0.5 Threshold

In [ ]:
threshold = 0.5

X_saudi = check_and_reorder_columns(X_nz, X_saudi)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz, y_nz)

# Predict the outcomes on the Saudi dataset using predict_proba
y_proba_saudi = xgb_model_nz.predict_proba(X_saudi)[:, 1]
y_pred_saudi = (y_proba_saudi >= threshold).astype(int)

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_saudi, y_pred_saudi)
sensitivity = recall_score(y_saudi, y_pred_saudi)
specificity = specificity_score(y_saudi, y_pred_saudi)
roc_auc = roc_auc_score(y_saudi, y_proba_saudi)

# Compute the standard deviations
conf_matrix = confusion_matrix(y_saudi, y_pred_saudi)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Threshold

In [ ]:
threshold = 0.3

X_saudi = check_and_reorder_columns(X_nz, X_saudi)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz, y_nz)

# Predict the outcomes on the Saudi dataset using predict_proba
y_proba_saudi = xgb_model_nz.predict_proba(X_saudi)[:, 1]
y_pred_saudi = (y_proba_saudi >= threshold).astype(int)

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_saudi, y_pred_saudi)
sensitivity = recall_score(y_saudi, y_pred_saudi)
specificity = specificity_score(y_saudi, y_pred_saudi)
roc_auc = roc_auc_score(y_saudi, y_proba_saudi)

# Compute the standard deviations
conf_matrix = confusion_matrix(y_saudi, y_pred_saudi)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

### Best-performing model - train on New Zealand and validate on Saudi

#### 0.5 Threshold

In [ ]:
threshold = 0.5

X_nz_4features = X_nz.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)
X_saudi_4features = X_saudi.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)

X_saudi_4features = check_and_reorder_columns(X_nz_4features, X_saudi_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz_4features, y_nz)

# Predict the outcomes on the Saudi dataset using predict_proba
y_proba_saudi = xgb_model_nz.predict_proba(X_saudi_4features)[:, 1]
y_pred_saudi = (y_proba_saudi >= threshold).astype(int)

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_saudi, y_pred_saudi)
sensitivity = recall_score(y_saudi, y_pred_saudi)
specificity = specificity_score(y_saudi, y_pred_saudi)
roc_auc = roc_auc_score(y_saudi, y_proba_saudi)  # Ensure y_saudi and predictions are appropriate for ROC calculation

# Compute the standard deviations
conf_matrix = confusion_matrix(y_saudi, y_pred_saudi)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### O.3 Threshold

In [ ]:
threshold = 0.3

X_nz_4features = X_nz.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)
X_saudi_4features = X_saudi.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)

X_saudi_4features = check_and_reorder_columns(X_nz_4features, X_saudi_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz_4features, y_nz)

# Predict the outcomes on the Saudi dataset using predict_proba
y_proba_saudi = xgb_model_nz.predict_proba(X_saudi_4features)[:, 1]
y_pred_saudi = (y_proba_saudi >= threshold).astype(int)

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_saudi, y_pred_saudi)
sensitivity = recall_score(y_saudi, y_pred_saudi)
specificity = specificity_score(y_saudi, y_pred_saudi)
roc_auc = roc_auc_score(y_saudi, y_proba_saudi)  # Ensure y_saudi and predictions are appropriate for ROC calculation

# Compute the standard deviations
conf_matrix = confusion_matrix(y_saudi, y_pred_saudi)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

## Saudi Dataset - XGB Model

### Full-feature model - train on Saudi dataset and test on New Zealand

#### O.5 Threshold

In [ ]:
threshold = 0.5

X_nz = check_and_reorder_columns(X_saudi, X_nz)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi, y_saudi)

# Predict the outcomes on the NZ dataset using predict_proba and threshold
y_proba_nz = xgb_model_saudi.predict_proba(X_nz)[:, 1]
y_pred_nz = (y_proba_nz >= threshold).astype(int)  # Using 0.5 as the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_nz, y_pred_nz)
sensitivity = recall_score(y_nz, y_pred_nz)
specificity = specificity_score(y_nz, y_pred_nz)
roc_auc = roc_auc_score(y_nz, y_proba_nz)  # Directly using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_nz, y_pred_nz)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Threshold

In [ ]:
threshold = 0.3

X_nz = check_and_reorder_columns(X_saudi, X_nz)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi, y_saudi)

# Predict the outcomes on the NZ dataset using predict_proba and threshold
y_proba_nz = xgb_model_saudi.predict_proba(X_nz)[:, 1]
y_pred_nz = (y_proba_nz >= threshold).astype(int)  # Using 0.5 as the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_nz, y_pred_nz)
sensitivity = recall_score(y_nz, y_pred_nz)
specificity = specificity_score(y_nz, y_pred_nz)
roc_auc = roc_auc_score(y_nz, y_proba_nz)  # Directly using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_nz, y_pred_nz)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

## 4-feature model

#### 0.5 Threshold

In [ ]:
threshold = 0.5

X_saudi_4features = X_saudi.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)
X_nz_4features = X_nz.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)

X_nz_4features = check_and_reorder_columns(X_saudi_4features, X_nz_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=400,
    max_depth=3,
    learning_rate=0.01,
    subsample=1,
    colsample_bytree=0.7,
    gamma=0.3,
    reg_lambda=3,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi_4features, y_saudi)

# Predict the outcomes on the NZ dataset using predict_proba and applying a threshold of 0.5
y_proba_nz = xgb_model_saudi.predict_proba(X_nz_4features)[:, 1]
y_pred_nz = (y_proba_nz >= threshold).astype(int)  # Using 0.5 as the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_nz, y_pred_nz)
sensitivity = recall_score(y_nz, y_pred_nz)
specificity = specificity_score(y_nz, y_pred_nz)
roc_auc = roc_auc_score(y_nz, y_proba_nz)  # Using the probabilities for ROC AUC calculation

# Compute the standard deviations
conf_matrix = confusion_matrix(y_nz, y_pred_nz)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Threshold

In [ ]:
threshold = 0.3

X_saudi_4features = X_saudi.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)
X_nz_4features = X_nz.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)

X_nz_4features = check_and_reorder_columns(X_saudi_4features, X_nz_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=400,
    max_depth=3,
    learning_rate=0.01,
    subsample=1,
    colsample_bytree=0.7,
    gamma=0.3,
    reg_lambda=3,
    reg_alpha=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi_4features, y_saudi)

# Predict the outcomes on the NZ dataset using predict_proba and applying a threshold of 0.5
y_proba_nz = xgb_model_saudi.predict_proba(X_nz_4features)[:, 1]
y_pred_nz = (y_proba_nz >= threshold).astype(int)  # Using 0.5 as the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_nz, y_pred_nz)
sensitivity = recall_score(y_nz, y_pred_nz)
specificity = specificity_score(y_nz, y_pred_nz)
roc_auc = roc_auc_score(y_nz, y_proba_nz)  # Using the probabilities for ROC AUC calculation

# Compute the standard deviations
conf_matrix = confusion_matrix(y_nz, y_pred_nz)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# Final Test on Separate Dataset

## New Zealand Models

### Full featured model test on Polish dataset

#### 0.5 Threshold

In [ ]:
threshold = 0.5

X_polish = check_and_reorder_columns(X_nz, X_polish)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz, y_nz)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_nz.predict_proba(X_polish)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Using 0.5 as the classification threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Threshold

In [ ]:
threshold = 0.3

X_polish = check_and_reorder_columns(X_nz, X_polish)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz, y_nz)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_nz.predict_proba(X_polish)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Using 0.3 as the classification threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

### 4-features model test on Polish dataset

#### 0.5 Threshold

In [ ]:
threshold = 0.5

X_nz_4features = X_nz.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)
X_polish_4features = X_polish.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)

X_polish_4features = check_and_reorder_columns(X_nz_4features, X_polish_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz_4features, y_nz)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_nz.predict_proba(X_polish_4features)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Using 0.5 as the classification threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Threshold

In [ ]:
threshold = 0.3

X_nz_4features = X_nz.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)
X_polish_4features = X_polish.drop(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A8_Score', 'A10_Score', 'age_months', 'gender', 'family_pdd'], axis=1)

X_polish_4features = check_and_reorder_columns(X_nz_4features, X_polish_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_nz = XGBClassifier(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.5,
    gamma=0.3,
    reg_lambda=3,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the NZ dataset
xgb_model_nz.fit(X_nz_4features, y_nz)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_nz.predict_proba(X_polish_4features)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Using 0.3 as the classification threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

## Saudi models

### Full feature test on Polish dataset

#### 0.5 Threshold

In [ ]:
threshold = 0.5

X_saudi = check_and_reorder_columns(X_polish, X_saudi)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi, y_saudi)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_saudi.predict_proba(X_polish)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Convert probabilities to 0 or 1 based on the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
conf_mat = confusion_matrix(y_polish, y_pred_polish)
print("Confusion Matrix:")
print(conf_mat)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

#### 0.3 Threshold

In [ ]:
threshold = 0.3

X_saudi = check_and_reorder_columns(X_polish, X_saudi)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi, y_saudi)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_saudi.predict_proba(X_polish)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Convert probabilities to 0 or 1 based on the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
conf_mat = confusion_matrix(y_polish, y_pred_polish)
print("Confusion Matrix:")
print(conf_mat)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

###  4 Feature Model on Saudi Dataset

In [ ]:
threshold = 0.5

# Drop age_months, gender, and family_pdd features from both datasets
X_saudi_4features = X_saudi.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)
X_polish_4features = X_polish.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)

# Reorder columns in X_saudi dataset
X_saudi_4features = check_and_reorder_columns(X_polish_4features, X_saudi_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi_4features, y_saudi)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_saudi.predict_proba(X_polish_4features)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Convert probabilities to 0 or 1 based on the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
threshold = 0.3

# Drop specific features from both datasets
X_saudi_4features = X_saudi.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)
X_polish_4features = X_polish.drop(['A10_Score', 'A7_Score', 'A5_Score', 'A4_Score', 'A3_Score', 'A1_Score', 'family_pdd', 'age_months', 'gender'], axis=1)

# Reorder columns in X_saudi dataset
X_saudi_4features = check_and_reorder_columns(X_polish_4features, X_saudi_4features)

# Initialize the XGBoost model with relevant hyperparameters
xgb_model_saudi = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.7,
    gamma=0,
    reg_lambda=3,
    reg_alpha=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model on the Saudi dataset
xgb_model_saudi.fit(X_saudi_4features, y_saudi)

# Predict the probability outcomes on the Polish dataset
y_proba_polish = xgb_model_saudi.predict_proba(X_polish_4features)[:, 1]
y_pred_polish = (y_proba_polish >= threshold).astype(int)  # Convert probabilities to 0 or 1 based on the threshold

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_polish, y_pred_polish)
sensitivity = recall_score(y_polish, y_pred_polish)
specificity = specificity_score(y_polish, y_pred_polish)
roc_auc = roc_auc_score(y_polish, y_proba_polish)  # Using the probability scores for ROC AUC

# Compute the standard deviations
conf_matrix = confusion_matrix(y_polish, y_pred_polish)
tn, fp, fn, tp = conf_matrix.ravel()
total = tn + fp + fn + tp
balanced_acc_std = np.sqrt((balanced_acc * (1 - balanced_acc)) / total)
sens_std = np.sqrt((sensitivity * (1 - sensitivity)) / total)
spec_std = np.sqrt((specificity * (1 - specificity)) / total)
roc_auc_std = roc_auc * (1 - roc_auc)

# Print the evaluation results with standard deviations
print(f"Balanced Accuracy: {balanced_acc:.2f} ± {balanced_acc_std:.2f}")
print(f"Sensitivity: {sensitivity:.2f} ± {sens_std:.2f}")
print(f"Specificity: {specificity:.2f} ± {spec_std:.2f}")
print(f"ROC-AUC: {roc_auc:.2f} ± {roc_auc_std:.2f}")

# Confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Display the confusion matrix graphically
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()